<a href="https://colab.research.google.com/github/mhered/robotica_patito_pycon2025/blob/main/notebooks/training_act2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤗 x 🦾: Training ACT with LeRobot Notebook

Welcome to the **LeRobot ACT training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `ACT` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `ACT` policy for 100,000 steps typically takes **about 1.5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer.

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


# ⚠️ Before running any cells, select A100 GPU runtime type (requires making a purchase, the cheapest option of 100 units pay as you go  for ~10€ will be plenty. 1-2hrs of training expected).

## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:07
🔁 Restarting kernel...


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [2]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

Cloning into 'lerobot'...
remote: Enumerating objects: 40338, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 40338 (delta 5), reused 0 (delta 0), pack-reused 40324 (from 2)
Receiving objects: 100% (40338/40338), 169.74 MiB | 15.42 MiB/s, done.
Resolving deltas: 100% (26409/26409), done.
Filtering content: 100% (45/45), 69.03 MiB | 14.12 MiB/s, done.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.2                 |       h39aace5_0          66 KB  conda-forge
    ca-ce

## More setup
This cell installs `tree` and the git credential helper

In [3]:
!apt-get -qq update && apt-get -qq install -y tree
!git config --global credential.helper store

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package tree.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../tree_2.0.2-1_amd64.deb ...
Unpacking tree (2.0.2-1) ...
Setting up tree (2.0.2-1) ...
Processing triggers for man-db (2.10.2-1) ...


## Login onto Weights & Biases and Huggingface Hub
The first cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging. Go to https://wandb.ai/authorize to obtain the API key (you need to sign up first if you haven't already) and paste it on the first cell when requested.

The second cell logs you into the Huggingface Hub to upload checkpoints.
Go to https://huggingface.co/settings/tokens to create a token (you need to sign up first) then create a token. I called it `collab-pycon`. Paste it in the second cell below. Note: for successive runs you need to click on invalidate & refresh the token and paste it again.

Say Yes to adding the token as git credential.

In [1]:
!wandb login

/usr/local/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
The token `collab-pycon` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved in your conf

## Training ACT with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=USER/REPO_NAME`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `mhered/recording_test`.

2. `--policy.type=act`:  
   Specifies the policy configuration to use. `act` refers to [configuration_act.py](../lerobot/common/policies/act/configuration_act.py), which will automatically adapt to your dataset’s setup (e.g., number of motors and cameras).

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

Note: If runtime disconnects before a checkpoint is reached and uploaded you cannot resume, lose progress is lost. By default the first checkpoint is fater 20k steps!!
Note: three versions of the train command are provided:
* one to launch training the first time.
* one to resume training when there is still a local `training_state`
* one to warm start from the weights in the last checkpoint saved in the Hub

In [11]:
# use this to warm start from saved Hub weights after adding more episodes. Here # steps is the number of steps to do
!cd lerobot && lerobot-train \
--dataset.repo_id=mhered/recording-test \
--policy.type=act \
--output_dir=outputs/train/act_so100_pato_test \
--job_name=act_so100_pato_test2 \
--policy.device=cuda \
--wandb.enable=true \
--policy.repo_id=mhered/my_act2 \
--policy.pretrained_path=mhered/my_act \
--steps=20000 \
--save_checkpoint=true \
--save_freq=2000 \
--eval_freq=2000 \
--policy.use_amp=true \
--num_workers=2 \
--policy.push_to_hub=false


INFO 2025-10-08 22:45:38 ot_train.py:148 {'batch_size': 8,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'ColorJitter',
                                                       'weight': 1.0},
                                          'hue': {'kwargs': {'hue': [-0.05,
                 

## Upload checkpoint to Hugging Face Hub
Now after training is done check the checkpoints created and upload the last checkpoint to the Hugging Face hub at https://huggingface.co/mhered/my_act2/tree/main

In [13]:
!tree /content/lerobot/outputs/train/act_so100_pato_test/checkpoints

/content/lerobot/outputs/train/act_so100_pato_test/checkpoints
├── 002000
│   ├── pretrained_model
│   │   ├── config.json
│   │   ├── model.safetensors
│   │   ├── policy_postprocessor.json
│   │   ├── policy_postprocessor_step_0_unnormalizer_processor.safetensors
│   │   ├── policy_preprocessor.json
│   │   ├── policy_preprocessor_step_3_normalizer_processor.safetensors
│   │   └── train_config.json
│   └── training_state
│       ├── optimizer_param_groups.json
│       ├── optimizer_state.safetensors
│       ├── rng_state.safetensors
│       └── training_step.json
├── 004000
│   ├── pretrained_model
│   │   ├── config.json
│   │   ├── model.safetensors
│   │   ├── policy_postprocessor.json
│   │   ├── policy_postprocessor_step_0_unnormalizer_processor.safetensors
│   │   ├── policy_preprocessor.json
│   │   ├── policy_preprocessor_step_3_normalizer_processor.safetensors
│   │   └── train_config.json
│   └── training_state
│       ├── optimizer_param_groups.json
│       ├── optimizer_

In [14]:
!huggingface-cli upload mhered/my_act2 \
/content/lerobot/outputs/train/act_so100_pato_test/checkpoints/last/pretrained_model \
--repo-type model --commit-message "Upload last checkpoint"

⚠️  Warning: 'huggingface-cli upload' is deprecated. Use 'hf upload' instead.
Start hashing 7 files.
Finished hashing 7 files.
Processing Files (0 / 0)      : |          |  0.00B /  0.00B            
New Data Upload               : |          |  0.00B /  0.00B            

  ...d_model/model.safetensors:   0% 562k/207M [00:00<?, ?B/s]

Processing Files (0 / 1)      :   0% 562k/207M [00:01<09:48, 350kB/s,  402kB/s  ]
New Data Upload               :   0% 562k/134M [00:01<06:21, 350kB/s,  402kB/s  ]

  ...d_model/model.safetensors:   0% 562k/207M [00:00<?, ?B/s]

Processing Files (0 / 1)      :   1% 1.12M/207M [00:02<05:29, 624kB/s,  622kB/s  ]
New Data Upload               :   1% 1.12M/134M [00:02<03:33, 624kB/s,  622kB/s  ]

  ...d_model/model.safetensors:   1% 1.12M/207M [00:00<03:41, 928kB/s] 

Processing Files (0 / 1)      :   1% 1.68M/207M [00:02<04:04, 837kB/s,  764kB/s  ]
New Data Upload               :   1% 1.68M/201M [00:02<03:58, 837kB/s,  764kB/s  ]

Processing Files (0 / 1)  

## Disconnect to avoid consuming compute

In [ ]:
from google.colab import runtime
runtime.unassign()